<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Create an optimal linear-phase low-pass filter `h` with a transition band of
[0.2, 0.3] (assuming a Nyquist frequency of 1):


In [ ]:
import numpy as np
from scipy.signal import remez, minimum_phase, freqz, group_delay
import matplotlib.pyplot as plt
freq = [0, 0.2, 0.3, 1.0]
desired = [1, 0]
h_linear = remez(151, freq, desired, fs=2)

Convert it to minimum phase:


In [ ]:
h_hil = minimum_phase(h_linear, method='hilbert')
h_hom = minimum_phase(h_linear, method='homomorphic')
h_hom_full = minimum_phase(h_linear, method='homomorphic', half=False)

Compare the impulse and frequency response of the four filters:


In [ ]:
fig0, ax0 = plt.subplots(figsize=(6, 3), tight_layout=True)
fig1, axs = plt.subplots(3, sharex='all', figsize=(6, 6), tight_layout=True)
ax0.set_title("Impulse response")
ax0.set(xlabel='Samples', ylabel='Amplitude', xlim=(0, len(h_linear) - 1))
axs[0].set_title("Frequency Response")
axs[0].set(xlim=(0, .65), ylabel="Magnitude / dB")
axs[1].set(ylabel="Phase / rad")
axs[2].set(ylabel="Group Delay / samples", ylim=(-31, 81),
            xlabel='Normalized Frequency (Nyqist frequency: 1)')
for h, lb in ((h_linear,   f'Linear ({len(h_linear)})'),
              (h_hil,      f'Min-Hilbert ({len(h_hil)})'),
              (h_hom,      f'Min-Homomorphic ({len(h_hom)})'),
              (h_hom_full, f'Min-Homom. Full ({len(h_hom_full)})')):
    w_H, H = freqz(h, fs=2)
    w_gd, gd = group_delay((h, 1), fs=2)

    alpha = 1.0 if lb == 'linear' else 0.5  # full opacity for 'linear' line
    ax0.plot(h, '.-', alpha=alpha, label=lb)
    axs[0].plot(w_H, 20 * np.log10(np.abs(H)), alpha=alpha)
    axs[1].plot(w_H, np.unwrap(np.angle(H)), alpha=alpha, label=lb)
    axs[2].plot(w_gd, gd, alpha=alpha)
ax0.grid(True)
ax0.legend(title='Filter Phase (Order)')
axs[1].legend(title='Filter Phase (Order)', loc='lower right')
for ax_ in axs:  # shade transition band:
    ax_.axvspan(freq[1], freq[2], color='y', alpha=.25)
    ax_.grid(True)
plt.show()

The impulse response and group delay plot depict the 75 sample delay of the linear
phase filter `h`. The phase should also be linear in the stop band--due to the small
magnitude, numeric noise dominates there. Furthermore, the plots show that the
minimum phase filters clearly show a reduced (negative) phase slope in the pass and
transition band. The plots also illustrate that the filter with parameters
``method='homomorphic', half=False`` has same order and magnitude response as the
linear filter `h` whereas the other minimum phase filters have only half the order
and the square root  of the magnitude response.
